<font color='darkorange'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, add your functions to the *apputil\.py* file and *app\.py* file as instructed. *These exercises use the same [Titanic dataset](https://www.kaggle.com/competitions/titanic/data) as the lab.*


## Exercise 1: Survival Patterns


For this exercise you will analyze survival patterns on the Titanic by looking at passenger class, sex, and age group. Name the function `survival_demographics()`.

1. Create a new column in the Titanic dataset that classifies passengers into age categories (i.e., a pandas `category` series). The categories should be:
    - Child (up to 12)
    - Teen (13–19)
    - Adult (20–59)
    - Senior (60+)  
  
	Hint: The `pd.cut()` function might come in handy here.

2. Group the passengers by class, sex, and age group.  

3. For each group, calculate:  
    - The total number of passengers, `n_passengers`
    - The number of survivors, `n_survivors`
    - The survival rate, `survival_rate`

4. Return a table that includes the results for *all* combinations of class, sex, and age group.  

5. Order the results so they are easy to interpret.  

6. Come up with a clear question that your results table makes you curious about (e.g., “Did women in first class have a higher survival rate than men in other classes?”). Write this question in your `app.py` file above the call to your visualization function, using `st.write("Your Question Here")`.
   
7. Create a Plotly visualization in a function named `visualize_demographic()` that directly addresses your question by returning a Plotly figure (e.g., `fig = px. ...`). You are free to choose the chart type that you think best communicates the findings. Be creative — try different approaches, compare them, and ensure that your chart clearly answers the question you posed.


In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

In [2]:
def survival_demographics(df):
    # Create age group categories
    age_bins = [0, 12, 19, 59, float('inf')]
    age_labels = ['Child', 'Teen', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=True)
    df['age_group'] = df['age_group'].astype('category')

    # Group by class, sex, and age group
    grouped = df.groupby(['Pclass', 'Sex', 'age_group'])

    # Aggregate stats
    summary = grouped.agg(
        n_passengers=('PassengerId', 'count'),
        n_survivors=('Survived', 'sum')
    ).reset_index()

    # Calculate survival rate
    summary['survival_rate'] = summary['n_survivors'] / summary['n_passengers']

    # Sort for readability
    summary = summary.sort_values(by=['Pclass', 'Sex', 'age_group']).reset_index(drop=True)

    return summary

In [3]:
import plotly.express as px

def visualize_demographic(summary_df):
    # Highlight adult women in first class
    highlight = (
        (summary_df['Pclass'] == 1) &
        (summary_df['Sex'] == 'female') &
        (summary_df['age_group'] == 'Adult')
    )
    summary_df['highlight'] = highlight

    fig = px.bar(
        summary_df,
        x='age_group',
        y='survival_rate',
        color='Sex',
        facet_col='Pclass',
        barmode='group',
        title='Survival Rate by Class, Sex, and Age Group',
        color_discrete_map={'male': 'steelblue', 'female': 'salmon'},
        hover_data=['n_passengers', 'n_survivors']
    )

    fig.update_traces(marker_line_width=1.5, marker_line_color='black')
    fig.update_layout(showlegend=True)

    return fig

In [4]:
import streamlit as st

# Load data
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

# Analyze demographics
summary_df = survival_demographics(df)

# Pose your question
st.write("Did adult women in first class have a higher survival rate than any other demographic group?")

# Show table
st.dataframe(summary_df)

# Show visualization
fig = visualize_demographic(summary_df)
st.plotly_chart(fig)

/tmp/ipykernel_11051/944355136.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(['Pclass', 'Sex', 'age_group'])
2025-09-28 16:49:04.045 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 16:49:04.193 
  command:

    streamlit run /root/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-28 16:49:04.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 16:49:04.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-28 16:49:04.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ign

DeltaGenerator()

## Exercise 2: Family Size and Wealth

Using the Titanic dataset, write a function named `family_groups()` to explore the relationship between family size, passenger class, and ticket fare.  

1. Create a new column in the Titanic dataset that represents the total family size for each passenger, `family_size`. Family size is defined as the number of siblings/spouses aboard plus the number of parents/children aboard, plus the passenger themselves.

2. Group the passengers by family size and passenger class. For each group, calculate:  
   - The total number of passengers, `n_passengers`
   - The average ticket fare, `avg_fare`
   - The minimum and maximum ticket fares (to capture variation in wealth), `min_fare` and `max_fare`

3. Return a table with these results, sorted so that the values are clear and easy to interpret (for example, by class and then family size).

4. Write a function called `last_names()` that extracts the last name of each passenger from the `Name` column, and returns the count for each last name (i.e., a pandas series with last name as index, and count as value). Does this result agree with that of the data table above? Share your findings in your app using `st.write`.

5. Just like you did in Exercise 1, come up with a clear question that your results makes you curious about. Write this question in your app.py file above the call to your visualization function. Then, create a Plotly visualization in a function named `visualize_families()` that directly addresses your question. As in Exercise 1 you are free to choose the chart type that you think best communicates the findings.

## Bonus Question

Add a new column, `older_passenger`, to the Titanic dataset that indicates whether each passenger’s age is above the median age for *their* passenger class. So, suppose row $x$ is in passenger class 2. Then, a value of `True` at row $x$ would indicate that passenger older than 50% of class 2 passengers, and `False` would indicate that they younger.

- You should use pandas functions to accomplish this.
- The new column should contain Boolean values (True if the age is above the median, False if less than or equal to).
- Return the updated table in the function `determine_age_division()`

Once you’ve created this column, consider how this age division relates to your analysis above. Try to visualize this analysis in Plotly using the function name `visualize_age_division()`.